In [ ]:
import re
from typing import Any

from panel.io.mime_render import exec_with_return
import pandas as pd
import panel as pn
import openai
from panel.widgets import ChatInterface, ChatEntry

pn.extension("perspective")

DATAFRAME_PROMPT = """
    Here are the columns in your DataFrame: {columns}.
    Create  a plot with hvplot that highlights an interesting
    relationship between the columns with hvplot groupby kwarg.
"""

CODE_REGEX = re.compile(r"```python(.*?)```", re.DOTALL)

USER = "User"
PLOT_ASSISTANT="Plot Assistant"
CODE_ASSISTANT="Code Assistant"

async def respond_with_openai(contents: Any):
    # extract the DataFrame
    if isinstance(contents, pd.DataFrame):
        global df
        df = contents
        columns = contents.columns
        message = DATAFRAME_PROMPT.format(columns=columns)
    else:
        message = contents

    # ask OpenAI to plot
    response = await openai.ChatCompletion.acreate(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": message}],
        temperature=0,
        max_tokens=500,
        stream=True,
    )
    message = ""
    async for chunk in response:
        message += chunk["choices"][0]["delta"].get("content", "")
        yield {"user": CODE_ASSISTANT, "value": message}


async def respond_with_executor(code: str):
    value = exec_with_return(code=code, global_context=globals())
    return ChatEntry(user=PLOT_ASSISTANT, value = value)


async def response_callback(
    contents: Any,
    user: str,
    instance: ChatInterface,
):
    if user == USER:
        async for chunk in respond_with_openai(contents):
            yield chunk
        instance.respond()
    elif user==CODE_ASSISTANT and CODE_REGEX.search(contents):
        yield await respond_with_executor(CODE_REGEX.search(contents).group(1))
    else:
        yield "Sorry. I don't know how to respond to this"

# https://raw.githubusercontent.com/holoviz/panel/main/examples/assets/occupancy.csv
chat_interface = ChatInterface(
    callback=response_callback, widgets=[pn.widgets.TextAreaInput(name="Text"), pn.widgets.FileInput(name="Upload")], height=800
)
chat_interface.servable()